In [1]:
# Run some setup code for this notebook.

import random
import numpy as np
import pandas as pd
# from cs231n.data_utils import load_CIFAR10
# import matplotlib.pyplot as plt

# # This is a bit of magic to make matplotlib figures appear inline in the notebook
# # rather than in a new window.
# %matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'

# # Some more magic so that the notebook will reload external python modules;
# # see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# %load_ext autoreload
# %autoreload 2

In [7]:
# Load the raw CIFAR-10 data.
data_dir = '/home/ubuntu/CS231N/data/split_datasets/'

X_train = pd.read_pickle(data_dir + "train_data.pkl").to_numpy()
y_train = pd.read_pickle(data_dir + "train_labels.pkl").to_numpy()
X_test = pd.read_pickle(data_dir + "test_data.pkl").to_numpy()
y_test = pd.read_pickle(data_dir + "test_labels.pkl").to_numpy()

y_train = y_train.flatten().astype(np.int64)
y_test = y_test.flatten().astype(np.int64)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (64000, 49152)
Training labels shape:  (64000,)
Test data shape:  (20000, 49152)
Test labels shape:  (20000,)


In [8]:
# Subsample the data for more efficient code execution in this exercise
num_training = 20000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 5000
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

# Reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

(20000, 49152) (5000, 49152)


In [11]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [12]:
neigh.score(X_test, y_test)

0.0328